In [1]:
import math
import warnings
warnings.filterwarnings("ignore")
import numpy as np
import pandas as pd
import medical_lib as ml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from matplotlib import pyplot
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_score
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from xgboost import plot_importance
from sklearn.model_selection import train_test_split

Einlesen der Trainingsdaten

In [4]:
traindata = pd.read_csv('naive_latest_todesinterval_model3_Train_langlebig.csv') 
traindata = traindata.iloc[:, 3:] 
traindata = traindata.sample(frac=1, random_state=42).reset_index(drop=True) #Shuffle Rows and Reset Index
train_features = traindata.iloc[:, :-1]
train_class = traindata.iloc[:, -1]

testdata = pd.read_csv('naive_latest_todesinterval_model3_Test_langlebig.csv') 
testdata = testdata.iloc[:, 3:] 
testdata = testdata.sample(frac=1, random_state=42).reset_index(drop=True)

test_features = testdata.iloc[:, :-1]
test_class = testdata.iloc[:, -1]
test_class_array = np.array(test_class)

result = pd.read_csv('automated_algorithmen.csv')
result = result.iloc[:, 1:]

KNN

In [5]:
medKNN = KNeighborsClassifier(n_neighbors=4)
medKNN.fit(train_features,train_class)
knnYpred = medKNN.predict(test_features)
accuracyKNN = accuracy_score(test_class, knnYpred)
precisionKNN = precision_score(test_class, knnYpred, average='weighted')
recallKNN = recall_score(test_class, knnYpred, average='weighted')
f1scoreKNN = f1_score(test_class, knnYpred, average='weighted')
print('KNN Accuracy: ', accuracyKNN, 'KNN Precision: ', precisionKNN, 'KNN Recall: ', recallKNN, 'KNN F1-Score: ', f1scoreKNN )
pred_tot_lebendigknn = []
actual_tot_lebendigknn = []
abweichungknn = []
for el in range(0, len(knnYpred)):
    dist = abs(knnYpred[el] - test_class_array[el])
    abweichungknn.append(dist)
    if knnYpred[el] < 7:
        pred_tot_lebendigknn.append(1)
    else: 
        pred_tot_lebendigknn.append(0)
    if test_class_array[el] < 7:
        actual_tot_lebendigknn.append(1)
    else:
        actual_tot_lebendigknn.append(0)
accuracyknn, precisionknn, recallknn, f1scoreknn = ml.scoring(pred_tot_lebendigknn, actual_tot_lebendigknn)
print('Tatsächlich: ', accuracyknn, precisionknn, recallknn, f1scoreknn)
print('Durchschnittliche Abweichung: ', np.mean(abweichungknn))
print('Standartabweichung der Abweichung: ', np.std(abweichungknn))
# index = result.index[result['Modellierung'] == 'Model_5_version_2'].tolist()
# result.at[index, 'KNN_precision'] = precisionknn
# result.at[index, 'KNN_recall'] = recallknn
# result.at[index, 'KNN_f1'] = f1scoreknn
# result.to_csv('automated_algorithmen.csv')

KNN Accuracy:  0.6953405017921147 KNN Precision:  0.8900000082795781 KNN Recall:  0.6953405017921147 KNN F1-Score:  0.7744503862150921
Tatsächlich:  0.7419354838709677 0.21428571428571427 0.75 0.3333333333333333
Durchschnittliche Abweichung:  0.8422939068100358
Standartabweichung der Abweichung:  1.3744479920296782
